In [1]:
import pandas as pd
import re
import plotly.offline as py
import plotly.graph_objs as go

py.init_notebook_mode(connected=True)

In [2]:
# Utility functions:
def join_names(x):
    out = ''.join(x)
    return re.sub("\s+", " ", out).strip()

def format_names_dict(orig_dict):
    return {re.sub("\s+", " ", key).strip(): orig_dict[key] for key in orig_dict.keys()}

def plot_bars(labels, data, name):
    py.iplot({
        "data": [go.Bar(
            x=labels,
            y=data
        )],
        "layout": go.Layout(title=name)
    })
    
def plot_multiple_bars(labels_list, data_list, trace_names, name, group=True):
    #layout = go.Layout(title=name, barmode='group') if group else go.Layout(title=name, barmode='group')  
    py.iplot({
        "data": [go.Bar(name=trace_names[i],
                       x=[num + 1 for num in range(len(labels_list[i]))],
                       y=data_list[i],
                       text=labels_list[i]) 
                 for i in range(len(labels_list))],
        "layout": go.Layout(
            title=name,
            barmode='group'
        )
    })

In [4]:
# Data preprocessing
# Column names for Salary Data
# ['NAME FIRST', 'NAME LAST', 'TITLE', 'DEPARTMENT', 'HIRE DATE', 'RACE', 
#   'GENDER', 'EMPLOYMENT TYPE', 'SALARY (FISCAL YEAR ALLOCATION)']

salaries_df = pd.read_csv("data/salaries.csv")
salaries_df['NAME'] = salaries_df[['NAME FIRST', 'NAME LAST']].apply(join_names, axis=1)
salaries_df['DEPARTMENT'] = salaries_df['DEPARTMENT'].apply(lambda x: x.strip())
salaries_df['SALARY'] = salaries_df['SALARY (FISCAL YEAR ALLOCATION)']

In [5]:
def most_paid(salaries_df, dept='all', gender='both'):
    out_df = salaries_df.sort_values(['SALARY (FISCAL YEAR ALLOCATION)'], 
                            ascending=False)[['NAME', 'GENDER', 'RACE', 'TITLE', 'HIRE DATE', 
                                              'DEPARTMENT', 'SALARY']]
    if gender =='female': 
        out_df = out_df.loc[out_df['GENDER'] == 'FEMALE'][['NAME', 'DEPARTMENT', 'TITLE', 'SALARY']]
    elif gender == 'male':
        out_df = out_df.loc[out_df['GENDER'] == 'MALE'][['NAME', 'DEPARTMENT', 'TITLE', 'SALARY']]
    else:
        out_df = out_df[['NAME', 'RACE', 'GENDER', 'HIRE DATE', 'DEPARTMENT', 'TITLE', 'SALARY']]

    if dept == 'all':
        return out_df
    else:
        return out_df.loc[out_df['DEPARTMENT'] == dept]

    
def most_paid_analysis():
    # Overall most paid employees
    mp_df = most_paid(salaries_df)
    mp_df_ten = mp_df.head(10)
    plot_bars(mp_df_ten['NAME'], mp_df_ten['SALARY'], "10 Most Paid Employees at UT")
    
    # Most paid employees by gender
    plot_multiple_bars([(mp_df.loc[mp_df['GENDER'] == 'FEMALE']).head(10)['NAME'], 
                        (mp_df.loc[mp_df['GENDER'] == 'MALE']).head(10)['NAME']],  
                       [(mp_df.loc[mp_df['GENDER'] == 'FEMALE']).head(10)['SALARY'], 
                        (mp_df.loc[mp_df['GENDER'] == 'MALE']).head(10)['SALARY']],
                       ["Female", "Male"],
                       "10 Most Paid Employees at UT by Gender")
    
    # Average employee pay by race
    #race_avg = (mp_df.groupby(['RACE'])['SALARY'].mean()).to_dict()
    race_med = (mp_df.groupby(['RACE'])['SALARY'].median()).to_dict()
    #race_med = {re.sub("\s+", " ", key).strip(): race_med[key] for key in race_med.keys()}
    race_med = format_names_dict(race_med)
    del race_med['']
    plot_bars(list(race_med.keys()), list(race_med.values()), "Median pay by race")
    
    # Top 10 jobs, filtering out Administrative jobs like President, Director, etc.
    # 
    mask = mp_df.groupby(['TITLE'])['NAME'].transform('count') > 20
    top_jobs_filtered = (mp_df[(mask)].groupby(['TITLE'])['SALARY']
                       .median().to_frame()
                       .filter(regex="^((?!President|Dean|Provost|Director).)*$", axis=0)
                       .sort_values(['SALARY'], ascending=False)
                       .head(10))
    top_jobs_filtered = format_names_dict(top_jobs_filtered.to_dict()['SALARY'])
    plot_bars(list(top_jobs_filtered.keys()), 
              list(top_jobs_filtered.values()), 
              "Top Ten Nonadministrative Jobs by Median Pay")
    
    # Top 10 Median employee pay by job
    job_avg = (mp_df.groupby(['TITLE'])['SALARY'].median())
    job_avg = job_avg.to_frame().sort_values(['SALARY'], ascending=False)
    top_jobs = job_avg.head(10).to_dict()['SALARY']
    top_jobs = format_names_dict(top_jobs)
    plot_bars(list(top_jobs.keys()), list(top_jobs.values()), "Top Ten Jobs by Median Pay")
    
    
    
    top_nonadmin_jobs = job_avg.filter(regex="^((?!President|Dean|Provost).)*$", axis=0).head(10).to_dict()['SALARY']
    top_nonadmin_jobs = format_names_dict(top_nonadmin_jobs)
    plot_bars(list(top_nonadmin_jobs.keys()), list(top_nonadmin_jobs.values()), "Top Ten Non-Admin Jobs by Median Pay")
    
    
most_paid_analysis()